In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the btuB gene
model0['lb_list']['EX_cbl1_e ']=0
model0['lb_list']['EX_cbl1_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([17,  5,  8,  7, 15, 14,  3, 20, 10,  8,  1, 16,  4,  4, 16,  5, 18,
        1, 16, 11, 17,  6,  7, 20,  5, 13, 16, 11, 19,  9]), 1: array([ 3,  2,  9, 18, 10,  1, 15,  8,  9, 18,  6, 14,  6, 10,  3,  4, 14,
       12, 16, 17, 13, 10,  2,  3, 20,  7, 10, 16, 10, 13])}
1
strain_number:  0 12.533333333333333
strain_various:  0 7.036729038094017
strain_number:  1 9.966666666666667
strain_various:  1 5.375768678885736
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([17,  5,  8,  7, 15, 14,  3, 20, 10,  8,  1, 16,  4,  4, 16,  5, 18,
        1, 16, 11, 17,  6,  7, 20,  5, 13, 16, 11, 19,  9]), 1: array([ 3,  2,  9, 18, 10,  1, 15,  8,  9, 18,  6, 14,  6, 10,  3,  4, 14,
       12, 16, 17, 13, 10,  2,  3, 20,  7, 10, 16, 10, 13])}
_2:  {0: [0.6784862515716307, 0.2857391744693639], 1: [0.6784862515716307, 0.2857391744693639], 2: [0.6784862515716307, 0.2857391744693639], 3: [0.6784862515716307, 0.2857391744693639], 4: [0.6784862515716307, 0.2857391744693639], 5

glc__D_e_1:  -14.684084941022675
glc__D_e_1:  -35.224746557157204
glc__D_e_1:  -0.5776951881681178
glc__D_e_1:  -0.4009154739700751
glc__D_e_1:  -11.7758786970802
glc__D_e_1:  -26.972610629683384
glc__D_e_1:  -0.8201535908330371
glc__D_e_1:  -8.151227293007278
glc__D_e_1:  -24.539416781073037
glc__D_e_1:  -0.5828451870842546
glc__D_e_1:  -9.164956129850133
strain_number:  0 45.6
strain_various:  0 19.260668039643207
strain_number:  1 1.2
strain_various:  1 0.7483314773547882
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 886.0797758742806, 349.4327818079753, 416.7191751231225, 437.1052129392963, 412.8297633912022, 397.79346092503994, 399.9679201075399, 415.5770701280515, 408.46768007940256, 429.0009086414891, 401.32293009752016, 334.54621235440334, 349.5117099423959]
slip_r:  349.5117099423959
15
glc__D_e_1:  -27.528513286360862
glc__D_e_1:  -20.723268637812822
glc__D_e_1:  -35.515687789436
glc__D_e_1:  -28.07366085036092
glc__D_e_1:  -20.956164850678018
glc__D_e_1:  -26.908381605601562
gl

glc__D_e_1:  -1.837755033167419
glc__D_e_1:  -1.9105559466222464
glc__D_e_1:  -2.1404436418475523
glc__D_e_1:  -0.9139031132834514
glc__D_e_1:  -2.3990394724219
glc__D_e_1:  -1.2538694427238866
glc__D_e_1:  -1.6772359096649545
glc__D_e_1:  -2.2051087048341875
glc__D_e_1:  -2.3765325299916187
glc__D_e_1:  -2.353705122787934
glc__D_e_1:  -3.7016207267562615
glc__D_e_1:  -1.2739569384601648
glc__D_e_1:  -3.6192348199081965
glc__D_e_1:  -5.8114921742352195
glc__D_e_1:  -1.7144238873583406
glc__D_e_1:  -1.5197453309487852
glc__D_e_1:  -1.9287825720084877
glc__D_e_1:  -1.1853702170188156
glc__D_e_1:  -0.6166431309382162
glc__D_e_1:  -0.8346193153269915
glc__D_e_1:  -0.8014377389599476
glc__D_e_1:  -0.8149329395422561
glc__D_e_1:  -1.2083427851168786
glc__D_e_1:  -1.8999149858743207
glc__D_e_1:  -0.6677229484231519
glc__D_e_1:  -0.10835837174642537
glc__D_e_1:  -0.5928099395971766
strain_number:  0 2.966666666666667
strain_various:  0 1.9913702708325125
strain_number:  1 0.0
strain_various:  

In [7]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 4, 11,  3, 11,  8,  3,  3,  3,  9, 19,  3,  6,  4,  5, 17, 18, 17,
        6, 12, 14,  3, 19,  5, 16, 12, 13, 17, 11, 12,  1]), 1: array([20, 18, 12, 14, 15, 13, 15, 14,  6, 13,  6,  9,  2,  6,  4, 16,  7,
       16,  6,  3, 12,  1,  8, 17, 13, 12, 13,  5,  9,  8])}
1
strain_number:  0 10.933333333333334
strain_various:  0 6.826093725957442
strain_number:  1 10.433333333333334
strain_various:  1 4.991214503727748
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 4, 11,  3, 11,  8,  3,  3,  3,  9, 19,  3,  6,  4,  5, 17, 18, 17,
        6, 12, 14,  3, 19,  5, 16, 12, 13, 17, 11, 12,  1]), 1: array([20, 18, 12, 14, 15, 13, 15, 14,  6, 13,  6,  9,  2,  6,  4, 16,  7,
       16,  6,  3, 12,  1,  8, 17, 13, 12, 13,  5,  9,  8])}
_2:  {0: [0.6459265309332798, 0.2857391744693639], 1: [0.6459265309332798, 0.2857391744693639], 2: [0.6459265309332798, 0.2857391744693639], 3: [0.6459265309332798, 0.2857391744693639], 4: [0.6459265309332798, 0.2857391744693639], 

glc__D_e_1:  -26.21593153603864
glc__D_e_1:  -15.533487671862812
glc__D_e_1:  -40.161466361948285
glc__D_e_1:  -12.712829882134637
glc__D_e_1:  -10.36395324630781
glc__D_e_1:  -12.613158383571305
glc__D_e_1:  -28.656708997193228
glc__D_e_1:  -0.19392606339701146
glc__D_e_1:  -8.12386898100079
glc__D_e_1:  -25.003494225152345
glc__D_e_1:  -12.860661523900653
glc__D_e_1:  -18.25752956933342
glc__D_e_1:  -0.02847213119205727
glc__D_e_1:  -21.703541420866298
glc__D_e_1:  -36.08833743639488
glc__D_e_1:  -0.13506874752008913
glc__D_e_1:  -21.276903459182293
strain_number:  0 35.43333333333333
strain_various:  0 16.92864108216867
strain_number:  1 1.2
strain_various:  1 1.9043809142780934
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 587.3181878750764, 355.20640202161104, 373.74348568098566, 403.9887261860947, 417.7691715405338, 441.86041926921905, 464.48565106883433, 426.39719943798906, 396.6634943810643, 440.11877359087026, 380.8002958293503, 377.1138249543776, 359.5236255843964, 372.215453913

glc__D_e_1:  -8.736105204215713
glc__D_e_1:  -13.47415879445698
glc__D_e_1:  -0.3437216407808883
glc__D_e_1:  -11.398893330872468
glc__D_e_1:  -4.922207345814931
glc__D_e_1:  -7.0096638759821435
glc__D_e_1:  -4.597858924735925
glc__D_e_1:  -3.4421941098625175
glc__D_e_1:  -2.8253205323457293
glc__D_e_1:  -2.4637784640137594
glc__D_e_1:  -0.9170259213140108
glc__D_e_1:  -0.6422123408796607
glc__D_e_1:  -0.3545871569400346
glc__D_e_1:  -0.7948939772397624
glc__D_e_1:  -1.3275398385041475
strain_number:  0 5.033333333333333
strain_various:  0 5.67146855369538
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 587.3181878750764, 355.20640202161104, 373.74348568098566, 403.9887261860947, 417.7691715405338, 441.86041926921905, 464.48565106883433, 426.39719943798906, 396.6634943810643, 440.11877359087026, 380.8002958293503, 377.1138249543776, 359.5236255843964, 372.21545391318756, 307.97108858677683, 289.7766853428478, 286.14840086495013, 220.9077960756726

In [6]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10,  8,  6,  9,  4, 10,  6,  9,  6,  9,  5,  2,  5, 11,  5,  8,  6,
       14, 13,  8,  2,  5, 17,  1, 16, 14, 16,  7, 11,  2]), 1: array([11,  4,  8,  8, 11, 18,  7,  4, 15,  1, 14, 14,  1,  3, 14,  4, 19,
       13,  7,  3, 11,  7, 17, 17, 18,  7, 15,  2,  4, 12])}
1
strain_number:  0 9.433333333333334
strain_various:  0 5.136038248386482
strain_number:  1 9.633333333333333
strain_various:  1 5.564670300705654
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10,  8,  6,  9,  4, 10,  6,  9,  6,  9,  5,  2,  5, 11,  5,  8,  6,
       14, 13,  8,  2,  5, 17,  1, 16, 14, 16,  7, 11,  2]), 1: array([11,  4,  8,  8, 11, 18,  7,  4, 15,  1, 14, 14,  1,  3, 14,  4, 19,
       13,  7,  3, 11,  7, 17, 17, 18,  7, 15,  2,  4, 12])}
_2:  {0: [0.5207337757122594, 0.2857391744693639], 1: [0.5207337757122594, 0.2857391744693639], 2: [0.5207337757122594, 0.2857391744693639], 3: [0.5207337757122594, 0.2857391744693639], 4: [0.5207337757122594, 0.2857391744693639], 5:

glc__D_e_1:  -19.91674683190099
glc__D_e_1:  -20.626156681227894
glc__D_e_1:  -17.832367302157667
glc__D_e_1:  -19.552010325443867
glc__D_e_1:  -8.835661852005744
glc__D_e_1:  -18.11707496841554
glc__D_e_1:  -9.909393645352102
glc__D_e_1:  -15.141099147489115
glc__D_e_1:  -10.633736217113688
glc__D_e_1:  -47.103202052713534
glc__D_e_1:  -22.892125894147426
strain_number:  0 40.833333333333336
strain_various:  0 15.87678248960482
strain_number:  1 0.03333333333333333
strain_various:  1 0.1795054935711501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 844.4572198956872, 351.25067817889027, 340.1810174474647, 435.4940213295477, 483.9915709290709, 457.3497585747587, 436.4115672339357, 510.8122140050353, 461.62183890670735, 436.920406762512, 461.6072529235262, 465.9247807644949, 425.36783252626356, 466.51025308030853, 336.85813744047823]
slip_r:  336.85813744047823
17
glc__D_e_1:  -2.787453097260645
glc__D_e_1:  -0.4139793995644998
glc__D_e_1:  -5.597683291307693
glc__D_e_1:  -19.93656606348018

glc__D_e_1:  -0.4866321754820485
glc__D_e_1:  -0.4609819279946594
glc__D_e_1:  -0.7061722387406616
glc__D_e_1:  -0.4592327913263494
glc__D_e_1:  -0.12837855950228028
glc__D_e_1:  -0.8245778560989928
glc__D_e_1:  -0.5530936155721463
glc__D_e_1:  -1.5943496830022734
strain_number:  0 6.733333333333333
strain_various:  0 6.366230351541553
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 844.4572198956872, 351.25067817889027, 340.1810174474647, 435.4940213295477, 483.9915709290709, 457.3497585747587, 436.4115672339357, 510.8122140050353, 461.62183890670735, 436.920406762512, 461.6072529235262, 465.9247807644949, 425.36783252626356, 466.51025308030853, 336.85813744047823, 190.71060246684524, 161.21832698197397, 173.1555925789538, 177.7897899452424, 205.63776770516014, 187.91069712357904]
slip_r:  187.91069712357904
23
glc__D_e_1:  -2.355404182534382
glc__D_e_1:  -0.505218925378093
glc__D_e_1:  -1.906621788616226
glc__D_e_1:  -2.585023566020796
glc__D_e

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 17,  2, 20,  9,  6, 11,  2,  5, 20, 18,  2,  2, 20,  1, 12, 10,
       16,  9,  2, 12,  8,  8, 19,  6, 16,  2,  9,  3,  9]), 1: array([15,  3, 11, 13, 12, 12, 10, 16,  2,  1,  8, 16,  8, 16, 16, 10, 14,
       10,  5,  2, 20, 20,  8, 10,  6,  1, 20,  6,  9, 14])}
1
strain_number:  0 10.766666666666667
strain_various:  0 7.61876338755546
strain_number:  1 10.466666666666667
strain_various:  1 5.542161632030913
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 17,  2, 20,  9,  6, 11,  2,  5, 20, 18,  2,  2, 20,  1, 12, 10,
       16,  9,  2, 12,  8,  8, 19,  6, 16,  2,  9,  3,  9]), 1: array([15,  3, 11, 13, 12, 12, 10, 16,  2,  1,  8, 16,  8, 16, 16, 10, 14,
       10,  5,  2, 20, 20,  8, 10,  6,  1, 20,  6,  9, 14])}
_2:  {0: [0.5090172841115191, 0.2857391744693639], 1: [0.5090172841115191, 0.2857391744693639], 2: [0.5090172841115191, 0.2857391744693639], 3: [0.5090172841115191, 0.2857391744693639], 4: [0.5090172841115191, 0.2857391744693639], 5

glc__D_e_1:  -2.3692292333606204
glc__D_e_1:  -0.0020963500200372742
glc__D_e_1:  -3.123304848495927
glc__D_e_1:  -17.675226348907266
glc__D_e_1:  -0.7977328551337237
glc__D_e_1:  -16.73085450328033
glc__D_e_1:  -11.207340027289156
glc__D_e_1:  -14.458458560598734
glc__D_e_1:  -14.219070883509648
glc__D_e_1:  -0.37738432150920076
glc__D_e_1:  -18.824866276276694
glc__D_e_1:  -4.0682126634024485
glc__D_e_1:  -0.1831763141753182
glc__D_e_1:  -8.467359769429734
strain_number:  0 46.333333333333336
strain_various:  0 14.44145268162298
strain_number:  1 1.0666666666666667
strain_various:  1 1.0624918300339485
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 951.0203612177297, 384.30687643110247, 370.15707209457213, 398.5553142033406, 423.7086028007081, 407.67581541265747, 440.4523383049699, 411.3631215511521, 438.9737417056678, 423.5255474783388, 384.66540412453844, 384.73829764913705, 381.3452243290751, 412.1961553716712]
slip_r:  412.1961553716712
16
glc__D_e_1:  -17.193580958735375
glc__D_e_1:

glc__D_e_1:  -0.9345973816830049
glc__D_e_1:  -1.1073693691256776
glc__D_e_1:  -0.89453101688118
glc__D_e_1:  -1.6747066519026106
glc__D_e_1:  -2.377881142079485
glc__D_e_1:  -0.7375599720337931
glc__D_e_1:  -3.215327400627003
glc__D_e_1:  -1.705786211034507
glc__D_e_1:  -0.47642506108413896
glc__D_e_1:  -1.1802833500116574
glc__D_e_1:  -0.6479600366689109
glc__D_e_1:  -0.9838011684636978
glc__D_e_1:  -3.054199649440263
glc__D_e_1:  -1.1942245473527198
glc__D_e_1:  -1.4556892626519842
glc__D_e_1:  -2.8237114563987125
glc__D_e_1:  -2.5793928792706007
glc__D_e_1:  -1.9205881493422865
glc__D_e_1:  -2.0588468979839107
glc__D_e_1:  -1.5947334049445017
glc__D_e_1:  -0.30489126727326443
glc__D_e_1:  -0.7883611765180668
glc__D_e_1:  -1.7907184710302073
glc__D_e_1:  -0.46575911424936556
glc__D_e_1:  -0.2187267858270332
glc__D_e_1:  -2.2098976149020935
glc__D_e_1:  -3.388045983285086
glc__D_e_1:  -5.846660907416414
glc__D_e_1:  -4.024626852020286
strain_number:  0 3.2
strain_various:  0 1.886796

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 6,  9,  5,  9, 17, 17, 16,  7, 14, 20, 10,  3, 11,  8, 18, 16, 20,
       12, 19, 12, 19, 13,  2,  9, 20, 14, 12,  3,  1,  6]), 1: array([10,  6, 11,  1, 15,  7,  6,  4, 10,  3,  1, 10, 11, 17, 10, 16, 16,
        3,  5,  9, 14, 11, 19,  6,  4, 10, 11, 13,  1, 12])}
1
strain_number:  0 13.5
strain_various:  0 6.907725144117032
strain_number:  1 9.066666666666666
strain_various:  1 4.932432350158377
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 6,  9,  5,  9, 17, 17, 16,  7, 14, 20, 10,  3, 11,  8, 18, 16, 20,
       12, 19, 12, 19, 13,  2,  9, 20, 14, 12,  3,  1,  6]), 1: array([10,  6, 11,  1, 15,  7,  6,  4, 10,  3,  1, 10, 11, 17, 10, 16, 16,
        3,  5,  9, 14, 11, 19,  6,  4, 10, 11, 13,  1, 12])}
_2:  {0: [0.5090172841115191, 0.2857391744693639], 1: [0.5090172841115191, 0.2857391744693639], 2: [0.5090172841115191, 0.2857391744693639], 3: [0.5090172841115191, 0.2857391744693639], 4: [0.5090172841115191, 0.2857391744693639], 5: [0.509017284

glc__D_e_1:  -34.22179605527613
glc__D_e_1:  -32.780366089531974
glc__D_e_1:  -21.419683020259246
glc__D_e_1:  -17.781230303362143
glc__D_e_1:  -10.498303263362393
glc__D_e_1:  -17.67703223108269
glc__D_e_1:  -6.580649359336867
glc__D_e_1:  -19.61014341046419
glc__D_e_1:  -0.5460724023715908
glc__D_e_1:  -8.38057584836367
glc__D_e_1:  -9.51245610555562
glc__D_e_1:  -21.791823280440475
glc__D_e_1:  -25.03475898673483
glc__D_e_1:  -23.92824199085228
glc__D_e_1:  -12.286644314022318
strain_number:  0 40.56666666666667
strain_various:  0 18.874468351600143
strain_number:  1 0.2
strain_various:  1 0.40000000000000013
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 768.698382177678, 411.07067932067935, 426.3416473877, 474.23876771926444, 497.2925215886791, 458.2628627313575, 499.48278341442625, 469.54345563943923, 441.96749525177967, 417.1900843378165, 414.73568338696566, 420.6784014779756, 264.2377918591627]
slip_r:  264.2377918591627
15
glc__D_e_1:  -18.311094831644695
glc__D_e_1:  -13.15512918

glc__D_e_1:  -1.8195386932312752
glc__D_e_1:  -0.9567655965932361
glc__D_e_1:  -1.667270353658653
glc__D_e_1:  -1.7568239821422804
glc__D_e_1:  -0.7211035380642021
glc__D_e_1:  -0.46526102140726544
glc__D_e_1:  -0.17736009345082926
glc__D_e_1:  -0.2579285704719719
glc__D_e_1:  -0.5161363156799601
glc__D_e_1:  -0.6536978278926014
glc__D_e_1:  -0.8908081189662472
glc__D_e_1:  -3.012822288815895
glc__D_e_1:  -2.185853965104534
glc__D_e_1:  -4.510593745215187
glc__D_e_1:  -10.441935302734626
glc__D_e_1:  -15.266111628124149
glc__D_e_1:  -29.071088108687842
strain_number:  0 4.7
strain_various:  0 7.793800955460264
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 768.698382177678, 411.07067932067935, 426.3416473877, 474.23876771926444, 497.2925215886791, 458.2628627313575, 499.48278341442625, 469.54345563943923, 441.96749525177967, 417.1900843378165, 414.73568338696566, 420.6784014779756, 264.2377918591627, 238.60936745350642, 197.30189327679113, 203.5

In [8]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([17, 16, 11, 20, 20,  6,  4,  7,  5,  5, 19,  9, 20, 18, 20, 17, 18,
       16, 18, 12,  2,  8,  3, 15, 10, 15,  9, 12, 19,  2]), 1: array([ 1, 20, 11,  7,  2, 12,  6,  5,  7, 15, 10, 18,  1, 12, 19,  1, 16,
       18,  3, 10,  5, 20, 13, 18,  6,  5,  8, 12,  2, 11])}
1
strain_number:  0 14.566666666666666
strain_various:  0 7.310646726217562
strain_number:  1 9.8
strain_various:  1 6.068497892120147
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([17, 16, 11, 20, 20,  6,  4,  7,  5,  5, 19,  9, 20, 18, 20, 17, 18,
       16, 18, 12,  2,  8,  3, 15, 10, 15,  9, 12, 19,  2]), 1: array([ 1, 20, 11,  7,  2, 12,  6,  5,  7, 15, 10, 18,  1, 12, 19,  1, 16,
       18,  3, 10,  5, 20, 13, 18,  6,  5,  8, 12,  2, 11])}
_2:  {0: [0.5090172841115191, 0.2857391744693639], 1: [0.5090172841115191, 0.2857391744693639], 2: [0.5090172841115191, 0.2857391744693639], 3: [0.5090172841115191, 0.2857391744693639], 4: [0.5090172841115191, 0.2857391744693639], 5: [0.509017284

glc__D_e_1:  -20.96693390533095
glc__D_e_1:  -6.078862431472418
glc__D_e_1:  -17.289798719023576
glc__D_e_1:  -15.613061637487615
glc__D_e_1:  -23.78854860464507
glc__D_e_1:  -28.69658075618893
glc__D_e_1:  -23.274199870649
glc__D_e_1:  -9.414307663946186
glc__D_e_1:  -36.27421897270923
glc__D_e_1:  -0.5132098398646114
glc__D_e_1:  -23.641075714709586
glc__D_e_1:  -20.25349168104272
glc__D_e_1:  -19.28722413880349
glc__D_e_1:  -16.090006501038097
glc__D_e_1:  -5.826139581960337
glc__D_e_1:  -6.426847167940963
glc__D_e_1:  -8.012098629699226
glc__D_e_1:  -5.34523334998186
glc__D_e_1:  -15.304413393299997
glc__D_e_1:  -11.554366031980578
glc__D_e_1:  -33.6748034462852
glc__D_e_1:  -22.87059084263564
glc__D_e_1:  -0.9879798845186214
glc__D_e_1:  -19.870873040748585
glc__D_e_1:  -30.596329545859014
glc__D_e_1:  -2.732401052836593
glc__D_e_1:  -21.72800460724006
glc__D_e_1:  -11.708621396854706
glc__D_e_1:  -2.240089863492374
glc__D_e_1:  -15.444540708499511
glc__D_e_1:  -5.715169713765572


glc__D_e_1:  -2.0177586827576572
glc__D_e_1:  -6.791064015347475
glc__D_e_1:  -4.098934782437342
strain_number:  0 3.8666666666666667
strain_various:  0 2.539466260632121
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 900.9373072103336, 350.80150800005606, 388.08741912860927, 412.98789588547663, 466.62836998236804, 486.55115261760017, 369.9259845085583, 344.7600625519717, 391.43804962870615, 411.4303562515001, 381.65656367619357, 352.57187396845, 324.31758665009886, 248.2047597032162, 267.14706283953876, 220.23147277812768, 202.14524658348188, 220.02405927405925]
slip_r:  220.02405927405925
20
glc__D_e_1:  -0.07000943680976973
glc__D_e_1:  -1.1280447515542966
glc__D_e_1:  -0.8179395728208085
glc__D_e_1:  -1.261391689045931
glc__D_e_1:  -1.6777133540919658
glc__D_e_1:  -1.6059833711448852
glc__D_e_1:  -0.8475752727192647
glc__D_e_1:  -2.9754680685490245
glc__D_e_1:  -1.5230969183398206
glc__D_e_1:  -1.0548214140362797
glc__D_e_1:  -0.080535826374

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 5, 14, 11, 20, 13,  8,  6, 10, 11,  7, 13, 20, 18, 20,  5,  7,  8,
       10, 17, 19,  3, 11,  8,  5,  8, 11,  2, 20, 14, 14]), 1: array([12, 12, 14,  1, 18,  9, 16, 13,  5, 20, 14, 20,  8, 12, 19, 17, 20,
       17, 13,  7, 16,  6,  5,  3, 17, 11,  8, 16, 10, 17])}
1
strain_number:  0 13.166666666666666
strain_various:  0 6.408761364118829
strain_number:  1 12.533333333333333
strain_various:  1 5.270884387104523
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 5, 14, 11, 20, 13,  8,  6, 10, 11,  7, 13, 20, 18, 20,  5,  7,  8,
       10, 17, 19,  3, 11,  8,  5,  8, 11,  2, 20, 14, 14]), 1: array([12, 12, 14,  1, 18,  9, 16, 13,  5, 20, 14, 20,  8, 12, 19, 17, 20,
       17, 13,  7, 16,  6,  5,  3, 17, 11,  8, 16, 10, 17])}
_2:  {0: [0.49885752915702836, 0.2857391744693639], 1: [0.49885752915702836, 0.2857391744693639], 2: [0.49885752915702836, 0.2857391744693639], 3: [0.49885752915702836, 0.2857391744693639], 4: [0.49885752915702836, 0.28573917446936

glc__D_e_1:  -24.31697116871114
glc__D_e_1:  -0.4770985717435199
glc__D_e_1:  -1.4435524347138298
glc__D_e_1:  -20.431506788257664
glc__D_e_1:  -30.880417026001755
glc__D_e_1:  -1.6278314792866055
glc__D_e_1:  -1.1670898349801588
glc__D_e_1:  -20.675390938069
glc__D_e_1:  -26.12639814680433
glc__D_e_1:  -1.434852848724212
glc__D_e_1:  -1.5739143260243997
glc__D_e_1:  -34.61123091728348
glc__D_e_1:  -21.06700853709501
glc__D_e_1:  -2.456850237194926
glc__D_e_1:  -0.9219578763012661
glc__D_e_1:  -8.98263048939991
glc__D_e_1:  -38.34193116605411
glc__D_e_1:  -4.162308065343961
glc__D_e_1:  -1.6229361926406882
glc__D_e_1:  -13.846382802099404
glc__D_e_1:  -17.78176346068744
glc__D_e_1:  -2.2122986598773746
glc__D_e_1:  -0.7311280314847126
glc__D_e_1:  -5.669720147313301
glc__D_e_1:  -23.591032129982842
glc__D_e_1:  -3.277528427857762
glc__D_e_1:  -0.2709626801638178
glc__D_e_1:  -3.12816779781237
glc__D_e_1:  -11.744447409874033
glc__D_e_1:  -1.5876955588114923
glc__D_e_1:  -1.417808590486

glc__D_e_1:  -9.03993517503242
glc__D_e_1:  -7.490691466369398
glc__D_e_1:  -11.524912474156539
glc__D_e_1:  -7.051396069536282
glc__D_e_1:  -5.962773032483853
glc__D_e_1:  -5.721123505883389
glc__D_e_1:  -2.255174773366991
glc__D_e_1:  -2.7884267116208465
glc__D_e_1:  -1.8191264537980638
strain_number:  0 6.733333333333333
strain_various:  0 4.4865230289043305
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 692.0590462169409, 366.1684308338719, 343.0518595181907, 436.12678826591366, 482.89552063322776, 416.4136511202326, 454.34983045776335, 443.8851027202362, 466.7590473319437, 406.61823352119995, 371.09305825937616, 345.11389988339, 353.0786194484775, 453.9137707279608, 369.5777409838034, 329.1140941847149, 285.14393918857866]
slip_r:  285.14393918857866
19
glc__D_e_1:  -2.480193967149864
glc__D_e_1:  -1.4509004212234866
glc__D_e_1:  -3.0757246896431716
glc__D_e_1:  -1.721486918766714
glc__D_e_1:  -2.487555364018277
glc__D_e_1:  -0.389130747992